# 01 - Load in Patient Data

Prep Output Direction

In [ ]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/results/map_performance_tracking'

Import Data

In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/metadata/mergedCerebellumExtensive.csv'
sheet = None

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [ ]:
drop_list = ['roi_path', 'nih7_acute', 'study_id']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
data_df.columns

Set the parameters for dropping rows

In [ ]:
column = 'focal_cerebellum'  # The column you'd like to evaluate
condition = 'not'  # The condition to check ('equal', 'above', 'below', 'not')
value = 1 # The value to drop if found

In [ ]:
# data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

# 02 - Load in the Patient Maps

This uses the column in the above CSV which stored all of the file paths in it
- For example, from the base_directory, */tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [ ]:
file_column = 'roi_path'

In [ ]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
dv_df = GiiNiiFileImport(import_path=data_df[file_column], file_pattern=None, file_column=None, process_special_values=False).run()
dv_df

# 03 - Load Map to Check Damage Within

Import Region of Interest Masks

In [ ]:
base_directory = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/data/nettekoven_maps/conn'
shared_glob_pattern = '*.nii*'

In [ ]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
iv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=shared_glob_pattern).run()
iv_df

Extract Damage Scores Per Region of Interest

In [ ]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [ ]:
from calvin_utils.nifti_utils.damage_score_utils import DamageScorer
damage_scorer = DamageScorer(mask_path, dv_df, iv_df)
dmg_df = damage_scorer.calculate_damage_scores('avg_in_subject')
dmg_df = damage_scorer.sort_dataframes_by_index(dmg_df)
dmg_df

# 04 - Label Outcome Columns by Which Cluster they Belong To
- copy the output of the below cell and fill it out with categorical labels. 

In [ ]:
from calvin_utils.ccm_utils.symptom_specificity import get_column_labels
# get_column_labels(data_df)

Copy tge above dictionary if you want to organize your outcomes for prediction into categories. 
- Otherwise, leave label_dict = None.

Example of a full label_dict: 
```
label_dict = {
'MotorL_acute': 'motor',
'MotorR_acute': 'motor',
'bvmt_bias_acute': 'cognitive',
'bvmt_imt_acute': 'cognitive'
}
```

In [ ]:
label_dict = {
 'nih7_acute': 'ataxia',
 'nih7_3month': 'ataxia',
 'gdss_10_3month': 'mood',
 'gdss_2_3month': 'mood',
 'gdss_12_3month': 'mood',
 'animal_raw_acute': 'fluency',
 'hvlt_perc_acute': 'fluency',
 'hvlt_perc_3month': 'fluency',
 }
#   'nih7_1year': 'ataxia',

# 05 - Relate damage to symptom clusters

In [ ]:
data_df = data_df.fillna(0)
dmg_df = dmg_df.fillna(0)

In [ ]:
dmg_df

In [ ]:
from calvin_utils.ccm_utils.symptom_specificity import SpecificityAnalyzer
SpecificityAnalyzer(X=dmg_df, Y=data_df, y_labels=label_dict, 
                    correlation='spearman', method='permutation',
                    vectorize=False, out_dir=out_dir+'/permutation', absval=True).run(1000)

That's all

-Calvin